In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
INDEX_COL = 0
MY_USER_INDEX = 5 - 1
MY_USER_STRING = 'User '+ str(MY_USER_INDEX + 1)
ROUND_PRECISION = 3
NO_RATING = -1
KNN_VALUE = 4

# Задание 1

In [3]:
data = pd.read_csv('CSV/data.csv', index_col=INDEX_COL, delimiter=', ', engine='python')
context_day = pd.read_csv('CSV/context_day.csv', index_col=INDEX_COL, delimiter=', ', engine='python')
context_place = pd.read_csv('CSV/context_place.csv', index_col=INDEX_COL, delimiter=', ', engine='python')
movie_names = pd.read_csv('CSV/movie_names.csv', index_col=INDEX_COL, delimiter=', ', engine='python', header=None)

indexes = dict(users = data.index, movies = data.keys())
indexes

{'users': Index(['User 1', 'User 2', 'User 3', 'User 4', 'User 5', 'User 6', 'User 7',
        'User 8', 'User 9', 'User 10', 'User 11', 'User 12', 'User 13',
        'User 14', 'User 15', 'User 16', 'User 17', 'User 18', 'User 19',
        'User 20', 'User 21', 'User 22', 'User 23', 'User 24', 'User 25',
        'User 26', 'User 27', 'User 28', 'User 29', 'User 30', 'User 31',
        'User 32', 'User 33', 'User 34', 'User 35', 'User 36', 'User 37',
        'User 38', 'User 39', 'User 40'],
       dtype='object'),
 'movies': Index(['Movie 1', 'Movie 2', 'Movie 3', 'Movie 4', 'Movie 5', 'Movie 6',
        'Movie 7', 'Movie 8', 'Movie 9', 'Movie 10', 'Movie 11', 'Movie 12',
        'Movie 13', 'Movie 14', 'Movie 15', 'Movie 16', 'Movie 17', 'Movie 18',
        'Movie 19', 'Movie 20', 'Movie 21', 'Movie 22', 'Movie 23', 'Movie 24',
        'Movie 25', 'Movie 26', 'Movie 27', 'Movie 28', 'Movie 29', 'Movie 30'],
       dtype='object')}

In [4]:
data

Movie 1  Movie 2  Movie 3  Movie 4  Movie 5  Movie 6  Movie 7  \
User 1        -1        4        1        5       -1       -1       -1   
User 2         4        5        2        3        3        4       -1   
User 3         4        2        5        4        1        5       -1   
User 4         5       -1        2       -1       -1        2        4   
User 5         4        3        3        3        5        3        3   
User 6         2       -1        3        3        3        3        4   
User 7         4        1        4        4        1        4        5   
User 8        -1        5        5        3        3        3        3   
User 9         1        2        3        1        5        3        3   
User 10        5       -1        1       -1        3        3        4   
User 11       -1        5       -1        1        5        2        4   
User 12        5        4        4       -1        3       -1        5   
User 13        1        1        4        5        4        3        5   
User 14        3        5        4       -1        1        2        4   
User 15        3        5        3        4        4       -1        4   
User 16        3        3        5        3        2        2        5   
User 17        1        1       -1       -1        4        4       -1   
User 18        4        3        3        2        1        5        3   
User 19        4        3        5        5        4        4       -1   
User 20        2       -1        2        5       -1       -1        1   
User 21       -1        2        1        2        3        5        4   
User 22        2        4        4       -1        4       -1        4   
User 23        5        1       -1        5        5       -1       -1   
User 24        5        4        2       -1        2        1        3   
User 25        3        1       -1        4        2       -1        4   
User 26        1        5       -1        3       -1        2        4   
User 27        4        4        3       -1        4        5        3   
User 28       -1       -1        4        2        3        2        1   
User 29        3        2        4        2        2        2        4   
User 30        2        3        5        4        1        3        3   
User 31        3        5        4        5        1        3        3   
User 32        5        4        5        4        1        5        5   
User 33        5        3        4        4        2        1        3   
User 34       -1        5       -1       -1        1       -1        5   
User 35        4       -1        5        4        2        2        3   
User 36        3        2       -1        5        5        1        5   
User 37       -1        4       -1        3        1        4        5   
User 38        3       -1        1        4        2        3        4   
User 39        1       -1        5       -1        1        2        3   
User 40       -1        1        1        1        1       -1        4   

         Movie 8  Movie 9  Movie 10  ...  Movie 21  Movie 22  Movie 23  \
User 1         1        3        -1  ...         2         4        -1   
User 2         5        3         2  ...        -1         2        -1   
User 3         3        5         2  ...         4        -1         5   
User 4         3        2         3  ...        -1         2         2   
User 5         4        5         2  ...         4         5         2   
User 6         2       -1         1  ...         5        -1         1   
User 7         2        2        -1  ...         2         5         1   
User 8         3        1         2  ...        -1         3         1   
User 9         1        2         5  ...         3         2         3   
User 10        1        1         2  ...         4         1         4   
User 11        2        4        -1  ...         4        -1         3   
User 12        5        5         3  ...         1         4         1   
User 13     

In [5]:
data_myUser = data[lambda x: x!=-1].iloc[MY_USER_INDEX]
data_myUser.to_frame().head(30)           

User 5
Movie 1      4.0
Movie 2      3.0
Movie 3      3.0
Movie 4      3.0
Movie 5      5.0
Movie 6      3.0
Movie 7      3.0
Movie 8      4.0
Movie 9      5.0
Movie 10     2.0
Movie 11     3.0
Movie 12     2.0
Movie 13     NaN
Movie 14     5.0
Movie 15     3.0
Movie 16     2.0
Movie 17     2.0
Movie 18     3.0
Movie 19     1.0
Movie 20     5.0
Movie 21     4.0
Movie 22     5.0
Movie 23     2.0
Movie 24     NaN
Movie 25     3.0
Movie 26     5.0
Movie 27     1.0
Movie 28     NaN
Movie 29     5.0
Movie 30     NaN

In [6]:
my_user_avg_score = round(data_myUser.mean(),3)
print(MY_USER_STRING, "average score:", my_user_avg_score)

User 5 average score: 3.308


In [7]:
def get_sim(user):
    data_copy = data.copy()
    user_sim = {}
    for cur_user in indexes['users']:
        if cur_user != user:
            pair = data_copy.loc[[user,cur_user]].transpose()
            pair = pair[(pair[user] != NO_RATING) & (pair[cur_user] != NO_RATING)]
            pair_cur_user = pair[cur_user].to_numpy()
            pair_user = pair[user].to_numpy()
            sim = sum(pair_cur_user * pair_user) / (np.sqrt(sum(pair_user ** 2)) * np.sqrt(sum(pair_cur_user ** 2)))
            user_sim[cur_user] = round(sim, ROUND_PRECISION)
    return user_sim
sim_dict = get_sim(MY_USER_STRING)
sim_df = pd.DataFrame.from_dict(sim_dict, orient='index')[0]
sim_df = sim_df.sort_values(ascending = False)
sim_df.head(KNN_VALUE)


User 12    0.922
User 22    0.917
User 2     0.909
User 4     0.890
Name: 0, dtype: float64

In [8]:
avg_user_rating = data.copy()
avg_user_rating = round(avg_user_rating[lambda x: x!=-1].transpose().mean(), 3)
avg_user_rating

User 1     2.952
User 2     3.292
User 3     2.800
User 4     3.143
User 5     3.308
User 6     3.160
User 7     2.880
User 8     3.292
User 9     2.808
User 10    2.720
User 11    3.125
User 12    3.846
User 13    3.154
User 14    3.000
User 15    3.375
User 16    3.407
User 17    3.000
User 18    2.704
User 19    3.125
User 20    3.182
User 21    2.960
User 22    3.217
User 23    3.391
User 24    2.815
User 25    2.760
User 26    2.920
User 27    3.333
User 28    2.923
User 29    3.200
User 30    2.964
User 31    3.074
User 32    3.000
User 33    2.769
User 34    3.042
User 35    3.174
User 36    3.125
User 37    3.250
User 38    3.000
User 39    2.680
User 40    2.963
dtype: float64

In [9]:
myUser_unrated_movies = data.iloc[MY_USER_INDEX][lambda x: x == -1].index
myUser_unrated_movies

Index(['Movie 13', 'Movie 24', 'Movie 28', 'Movie 30'], dtype='object')

In [10]:
calculated_ratings = {}
for movie in myUser_unrated_movies:
    top = 0
    bottom = 0
    for user, value in sim_df[0:KNN_VALUE].items():
        if data.loc[user,movie] != NO_RATING:
            top += value * (data.loc[user, movie] - avg_user_rating[user])
            bottom += value
            
    calculated_ratings[movie] = round(my_user_avg_score + (top / bottom), 3)
print(calculated_ratings)

{'Movie 13': 4.521, 'Movie 24': 3.228, 'Movie 28': 2.934, 'Movie 30': 3.875}


# Задание 2

In [11]:
CONTEXT_DAYS = ['Sat', 'Sun']
CONTEXT_PLACE = 'h'
myUser_unrated_movies

Index(['Movie 13', 'Movie 24', 'Movie 28', 'Movie 30'], dtype='object')

In [12]:
context_ratings = {}
for movie in myUser_unrated_movies:
    top = 0
    bottom = 0
    for user, value in sim_df[0:KNN_VALUE].items():
        if data.loc[user,movie] != NO_RATING:
            if context_day.loc[user, movie] in CONTEXT_DAYS:
                top += value * (data.loc[user, movie] - avg_user_rating[user])
                bottom += value
            if context_place.loc[user, movie] == CONTEXT_PLACE:
                top += value * (data.loc[user, movie] - avg_user_rating[user])
                bottom += value
    if bottom == 0:
        context_ratings[movie] = round(my_user_avg_score + 0, 3)
    else:
        context_ratings[movie] = round(my_user_avg_score + (top / bottom), 3)

movie_recommendation = list(dict(sorted(context_ratings.items(), key=lambda item: item[1], reverse=True)[:1]).keys())[0]

Movie 30


In [13]:
file_dict = {
    'user': MY_USER_STRING,
    '1': calculated_ratings,
    '2': dict(movie_names.loc[movie_recommendation])[1],
}
with open('result.json', 'w') as fp:
          json.dump(file_dict, fp, indent=3)
file_dict

{'user': 'User 5',
 '1': {'Movie 13': 4.521,
  'Movie 24': 3.228,
  'Movie 28': 2.934,
  'Movie 30': 3.875},
 '2': 'The Green Mile'}